In [1]:
# 简单的有2个隐层的神经网络模型
# 输入层 784*bathch 
# 第一个隐藏层 有256个神经元 ==> W1:784*256 b1:256*1
# 第二个隐藏层 有128个神经元 ==> W2:256*128 b2:128*1
# 输出层 ==> out:128*10 b_out:10*1
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
mnist  = input_data.read_data_sets('./data', one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [9]:
# 神经网络结构参数的一些设置
n_hidden_1 = 256  # 隐藏层1
n_hidden_2 = 128  # 隐藏层2
n_input = 784   # 输入层
n_classes = 10  # 输出层

# 输出层和输入层接口
x = tf.placeholder("float",[None, n_input])
y = tf.placeholder("float",[None, n_classes])

stddev = 0.1
weights = {
    'w1':tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=stddev)),
    'w2':tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=stddev)),
    'out':tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=stddev))
}

biases = {
    'b1':tf.Variable(tf.random_normal([n_hidden_1])), # 这里的b值可以进行高斯初始化也可以进行0值初始化
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}
print("Network Ready")

Network Ready


In [10]:
# 前向传播
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['w1']), _biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['w2']), _biases['b2']))
    return (tf.matmul(layer_2, _weights['out']) + _biases['out'])  # 最后的输出没有sigmoid

In [12]:
# 预测函数
pred = multilayer_perceptron(x, weights, biases)

# 损失函数的定义
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optm = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost) # learning rate 设置为0.001
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

# 变量初始化操作
init = tf.global_variables_initializer()
print("Function Ready")

Function Ready


In [13]:
# 定义一些超参数
training_epochs = 20
batch_size = 100
display_step = 4

sess = tf.Session()
sess.run(init)

for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x:batch_xs, y:batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost =avg_cost / total_batch
    
    # disply
    if (epoch+1) % display_step == 0:
        print("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x:batch_xs, y:batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x:mnist.test.images, y:mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print("TEST ACCURACY: %.3f" % (test_acc))
print("OPTIMIZATION FINISHED")

Epoch: 003/020 cost: 2.275243405
TRAIN ACCURACY: 0.120
TEST ACCURACY: 0.157
Epoch: 007/020 cost: 2.242115716
TRAIN ACCURACY: 0.340
TEST ACCURACY: 0.308
Epoch: 011/020 cost: 2.205320895
TRAIN ACCURACY: 0.360
TEST ACCURACY: 0.438
Epoch: 015/020 cost: 2.162627060
TRAIN ACCURACY: 0.450
TEST ACCURACY: 0.516
Epoch: 019/020 cost: 2.111885239
TRAIN ACCURACY: 0.520
TEST ACCURACY: 0.553
OPTIMIZATION FINISHED
